In [51]:
from sportsipy.nhl.teams import Teams
from scipy.stats import pearsonr
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

print('Correlation(WinRatio,GoalDifferential)\n')

maxCorr= -1.1
minCorr= 1.1
minYear = 2021
maxYear = 2021
corrList = []
# https://towardsdatascience.com/web-scraping-nba-stats-4b4f8c525994
def getGa(abbr,year):
    url = 'https://www.hockey-reference.com/teams/' + abbr + '/' + str(year) + '.html'
    html = urlopen(url)
    soup = BeautifulSoup(html,features="lxml")
    
    # avoid the first header row
    rows = soup.findAll('tr')[1:]
    all_stats = [[td.getText() for td in rows[i].findAll('td')]for i in range(len(rows))]
    all_stats2 = [e for e in all_stats if e]
    all_stats3=[x for x in all_stats2 if len(x)>3]
    team_stats = all_stats3[0]
    if year > 2005:
        goals_against = team_stats[8]
    else:
        goals_against = team_stats[9]
    return(goals_against)

corrd = {}
for year in range(2006, 2021):
    if year == 2005:
        continue
    teams = list(Teams(year))
    statd = {}
    for i,team in enumerate(teams[0:len(teams)]):
        G = team.games_played
        WinPct = round(100*(team.wins+(.5*team.overtime_losses))/G,3)
        GF = team.goals_for
        GA = int(getGa(team.abbreviation,year))
        statd[i] = {'Team' : team.name,
                    'W' : team.wins,
                    'L' : team.losses,
                    'OTL' : team.overtime_losses,                
                    'G' : G,
                    'WinPct' : WinPct,
                    'GF' : GF,
                    'GA' : GA,
                    'GD' : GF-GA,
                    'GD_per' : round((GF-GA)/G,2)}  

    df = pd.DataFrame(statd)
    df = df.transpose()
    
    corr, _ = pearsonr(df['GD'].tolist(), df['WinPct'].tolist())
    corrList.append(corr)
    
    if maxCorr < minCorr:
        minCorr = corr
        maxCorr = corr
        minYear = year
        maxYear = year
        print('%s: %s (new min) (new max)' % (year,round(corr,3)))
    elif corr < minCorr:
        minCorr= corr
        minYear = year
        print('%s: %s (new min)' % (year,round(corr,3)))
    elif corr > maxCorr:
        maxCorr= corr
        maxYear = year
        print('%s: %s (new max)' % (year,round(corr,3)))
    else: 
        print('%s: %s' % (year,round(corr,3)))

print('MIN:%s (%s)' % (minYear,round(minCorr,3)))
print('MAX:%s (%s)' % (maxYear,round(maxCorr,3)))
print('AVG:%s' % (round(sum(corrList)/len(corrList),3)))

Correlation(WinRatio,GoalDifferential)

2006: 0.929 (new min) (new max)
2007: 0.943 (new max)
2008: 0.914 (new min)
2009: 0.956 (new max)
2010: 0.946
2011: 0.929
2012: 0.902 (new min)
2013: 0.92
2014: 0.967 (new max)
2015: 0.961
2016: 0.956
2017: 0.973 (new max)
2018: 0.982 (new max)
2019: 0.975
2020: 0.967
MIN:2012 (0.902)
MAX:2018 (0.982)
AVG:0.948


In [ ]:
'''
MIN:2012 (0.902)
MAX:2018 (0.982)
AVG:0.95
'''

In [47]:
year=2012
teams = list(Teams(year))
statd = {}

for i,team in enumerate(teams[0:len(teams)]):
    G = team.games_played
    WinPct = round(100*(team.wins+(.5*team.overtime_losses))/G,3)
    GF = team.goals_for
    GA = int(getGa(team.abbreviation,year))
    statd[i] = {'Team' : team.name,
                'W' : team.wins,
                'L' : team.losses,
                'OTL' : team.overtime_losses,                
                'G' : G,
                'WinPct' : WinPct,
                'GF' : GF,
                'GA' : GA,
                'GD' : GF-GA,
                'GD_per' : round((GF-GA)/G,2)} 
        
df = pd.DataFrame(statd)
df = df.transpose()

x = list(df['WinPct'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Iwin'] =  index

x = list(df['GD'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Igd'] =  index

df['Delta'] = df['Igd']-df['Iwin']
x = list(df['Delta'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Idel'] =  index

df['WINvs50'] = df['WinPct']-50.0
df = df[['Team','W','OTL','L','G','GF','GA','GD','WinPct','GD_per','Iwin','Igd','Delta','Idel','WINvs50']]

df = df.sort_values('W',ascending=False)
df_mdian = df.loc[abs(df['GD']) == min(abs((df['GD'])))]
df_mdian = df_mdian[abs(df_mdian['WINvs50']) == min(abs(df_mdian['WINvs50']))]
df_lucky = df[df['Delta'] == max(df['Delta'])].head(1)
df_nluck = df[df['Delta'] == min(df['Delta'])].head(1)
print('%s Luckiest,Median,Unluckiest Teams:' % year)
df_ext = pd.concat([df_lucky,pd.concat([df_mdian,df_nluck],sort=False)],sort=False)
print(df_ext.to_string(index=False))

2012 Luckiest,Median,Unluckiest Teams:
               Team   W OTL   L   G   GF   GA   GD  WinPct GD_per  Iwin  Igd  Delta  Idel WINvs50
   Florida Panthers  38  18  26  82  197  216  -19  57.317  -0.23    13   19      6     0   7.317
 Montreal Canadiens  31  16  35  82  207  214   -7  47.561  -0.09    27   14    -13    29  -2.439
 Montreal Canadiens  31  16  35  82  207  214   -7  47.561  -0.09    27   14    -13    29  -2.439


In [ ]:
'''
2012 Luckiest,Median,Unluckiest Teams: (most anomolous)
               Team   W OTL   L   G   GF   GA   GD  WinPct GD_per  Iwin  Igd  Delta  Idel WINvs50
   Florida Panthers  38  18  26  82  197  216  -19  57.317  -0.23    13   19      6     0   7.317
 Montreal Canadiens  31  16  35  82  207  214   -7  47.561  -0.09    27   14    -13    29  -2.439
 Montreal Canadiens  31  16  35  82  207  214   -7  47.561  -0.09    27   14    -13    29  -2.439

2018 Luckiest,Median,Unluckiest Teams: (most predictable)
                Team   W OTL   L   G   GF   GA  GD  WinPct GD_per  Iwin  Igd  Delta  Idel WINvs50
 Washington Capitals  49   7  26  82  256  238  18  64.024   0.22     5   12      7     0  14.024
     St. Louis Blues  44   6  32  82  223  222   1  57.317   0.01    17   18      1     5   7.317
   Los Angeles Kings  45   8  29  82  237  202  35  59.756   0.43    11    6     -5    28   9.756
'''

In [48]:
year=2012
teams = list(Teams(year))
statd = {}

for i,team in enumerate(teams[0:len(teams)]):
    G = team.games_played
    WinPct = round(100*(team.wins+(.5*team.overtime_losses))/G,3)
    GF = team.goals_for
    GA = int(getGa(team.abbreviation,year))
    statd[i] = {'Team' : team.name,
                'W' : team.wins,
                'L' : team.losses,
                'OTL' : team.overtime_losses,                
                'G' : G,
                'WinPct' : WinPct,
                'GF' : GF,
                'GA' : GA,
                'GD' : GF-GA,
                'GD_per' : round((GF-GA)/G,2)} 
        
df = pd.DataFrame(statd)
df = df.transpose()

x = list(df['WinPct'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Iwin'] =  index

x = list(df['GD'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Igd'] =  index

df['Delta'] = df['Igd']-df['Iwin']
x = list(df['Delta'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Idel'] =  index

df['WINvs50'] = df['WinPct']-50.0
df = df[['Team','W','L','G','GF','GA','GD','WinPct','GD_per','Iwin','Igd','Delta','Idel','WINvs50']]

df = df.sort_values(['WinPct'],ascending = (False))
print('%s NHL Season - Ranked by [Iwin] -> Ratio(Games Won:Total Games) [WinPct] \n' % year)
print(df.to_string(index=False))

df = df.sort_values('GD',ascending=False)
print('\n%s NHL Season - Ranked by [Igd] -> Goal Differential [GD] \n' % year)
print(df.to_string(index=False))

df = df.sort_values('Idel',ascending=True)
print('\n%s NHL Season - Ranked by luck [Idel] -> Iwin-Igd (Delta) \n' % year)
print(df.to_string(index=False))

2012 NFL Season - Ranked by [Iwin] -> Ratio(Games Won:Total Games) [WinPct] 

                  Team   W   L   G   GF   GA   GD  WinPct GD_per  Iwin  Igd  Delta  Idel WINvs50
     Vancouver Canucks  51  22  82  241  191   50  67.683   0.61     0    3      3     4  17.683
      New York Rangers  51  24  82  222  182   40  66.463   0.49     1    4      3     4  16.463
       St. Louis Blues  49  22  82  206  155   51  66.463   0.62     1    2      1    10  16.463
   Pittsburgh Penguins  51  25  82  273  218   55  65.854   0.67     3    1     -2    24  15.854
   Nashville Predators  48  26  82  232  205   27  63.415   0.33     4    7      3     4  13.415
   Philadelphia Flyers  47  26  82  260  225   35  62.805   0.43     5    6      1    10  12.805
     Detroit Red Wings  48  28  82  239  200   39  62.195   0.48     6    5     -1    19  12.195
         Boston Bruins  49  29  82  260  199   61  62.195   0.74     6    0     -6    28  12.195
     New Jersey Devils  48  28  82  216  205   11